In [1]:
!nvidia-smi
# Monitoring and managing NVIDIA GPU devices

Tue Mar 12 10:28:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
# transformers  is a library developed by hugging phase which provides pretrained models or utilities for NLP models
# Snentencepiece is an additional dependancies usied for text tokenization
# datasets- package provides by Hugging Phase for nlp which has wide rannge of datasets.
#sacrebleu - used for computing bleu scores, used to evaluate the quality of machine-transalted text
#rouge_score - commonly used in evaluating quality of machine generated text
# py7zr = used for  working with 7z archieves
# -q = suppresses the unnecessary output (quiet)

In [3]:
# !pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [4]:
from transformers import pipeline, set_seed
# pipe line uses ready to use NLP Pipelines for task like text summarisation and transaltion etc.
# Import set_seed for randomness for reproducibility
from datasets import load_dataset, load_from_disk
#dataset for loading dataset from various source.
#dataset for  loading saved datasets.
import matplotlib.pyplot as plt
#For creating visualization
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
#load metric is for importing evaluation metric
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
# AutoModelForSeq2SeqLM This class is specifically designed for loading
# sequence-to-sequence (seq2seq) models in the Hugging Face.
# Seq2seq models are neural network architectures commonly used for tasks
# such as machine translation, text summarization, and conversational agents.
# Autotokenizer will tokenize the data for various NLP tasks.

import nltk
from nltk.tokenize import sent_tokenize
# It uses pre-trained sentence tokenizers to split text into individual
# sentences based on punctuation and other linguistic cues

from tqdm import tqdm
# tqdm is a Python library for creating progress bars
# and displaying progress updates during iterative tasks.
import torch


nltk.download("punkt")
#"punkt" is an argument specifying the specific resource or dataset
#  to be downloaded. In this case, "punkt" refers to the pre-trained
# tokenizer models for tokenizing text into sentences and words


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from transformers import  AutoModelForSeq2SeqLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model check

In [7]:
!wget https://github.com/HARIKRISHNANLK/Datas/raw/master/summarizer-data.zip
!unzip summarizer-data.zip
# Unzipping the dataset
# !wget is used download from resources from a given url

--2024-03-12 10:30:26--  https://github.com/HARIKRISHNANLK/Datas/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/HARIKRISHNANLK/Datas/master/summarizer-data.zip [following]
--2024-03-12 10:30:26--  https://raw.githubusercontent.com/HARIKRISHNANLK/Datas/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.03s   

2024-03-12 10:30:27 (287 MB/s) - ‘summarizer-data.zip.1’ saved [7903594/7903594]

Archive

In [8]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum
# Loading the dataset from the disk into a variable

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]
print(f"Split lengths: {split_lengths}")
# Printing the length of test, train and validating
print(f"Features: {dataset_samsum['train'].column_names}")
# Printing the features name
print("\nDialogue:")
print(dataset_samsum["test"][1]["dialogue"])
# Printing test sample

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [10]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
    # The max length is 1024, Which means dialoughe text will be truncated if it exceeds the length
    #It affects processing efficiency

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
      # It tokenizes the summary

    return {
        'input_ids' : input_encodings['input_ids'],
        # input id is the key which these token ids are stored
        'attention_mask': input_encodings['attention_mask'],
        # The attention mask for the dialogue is a binary mask that indicates which tokens in the
        # input sequence should be attended to by the model and which ones should be ignored.

        # The attention mask ensures that the model focuses only on the relevant tokens and
        # ignores any padding tokens. This helps in efficient processing and ensures that the model
        # doesn't attend to irrelevant parts of the input sequence.
        'labels': target_encodings['input_ids']
    }


In [11]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)
# Mapping the function
# Batched true means function applied to the a  branch not one by one


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
dataset_samsum_pt["train"]
#Training the data

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [14]:
# Training

from transformers import DataCollatorForSeq2Seq
# DataCollator is a utility class used in NLP proxessing tasks
# especially in seq2seq modeliing


seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

# A Seq2Seq (Sequence-to-Sequence) model is a type of neural network architecture
# that is commonly used for tasks where the input and output are both
# variable-length sequences

In [15]:

from transformers import TrainingArguments, Trainer
# Training arguments - holds the all hyper parametres and
# setting related to a training model
# Trainer - This class is responsible for training the model
# based on the specified training arguments.

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    # output_dir Directory where the fine-tuned model is saved
    # num_train_epochs=1  ; This argument sets the number of times the
    # entire training dataset will be processed during training
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    # In Machine Learning, training data is often divided into
    # small smaller sets called batches. The model train one batch at a time.
    # Batch size is a parameter that impacts training speed, memory
    # usage and model performance
    weight_decay=0.01, logging_steps=10,
    # Weight decay acts as regulizer, penalizing large weights in the model.
    # This prevents overfitting, It improves generalization. it works well
    # with unseen data.
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    # This argument specifies the strategy used to determine when to evaluate
    # the model. Here, it's set to 'steps', indicating that evaluation will occur
    #  based on a specific number of training steps, regardless of how many epochs
    #  have been completed.
    gradient_accumulation_steps=16
    # This argument specifies the strategy used to determine when to evaluate
    # the model. Here, it's set to 'steps', indicating that evaluation will occur
    #  based on a specific number of training steps, regardless of how many epochs
    #  have been completed.
)

In [16]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  # model=model_pegasus: This argument specifies the pre-trained
                  # Pegasus model instance that will be fine-tuned for text summarization.
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  # tokenizer=tokenizer: This argument provides the tokenizer
                  # object used to process the text data. The tokenizer converts
                  # text into numerical representations suitable for the model.
                  # data_collator=seq2seq_data_collator: This argument specifies
                  # the data_collator function, responsible for preparing batches
                  # of data for training
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [17]:
trainer.train()
# initiates the training process

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 151.694, 'train_samples_per_second': 5.399, 'train_steps_per_second': 0.336, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 1.0})

In [18]:
# Evaluating the performance of the dataset

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
  #  The provided code defines a function named calculate_metric_on_test_ds
  #  that's used to evaluate the performance of a text summarization
  #  model on a test dataset.
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        # tqdm is a popular pyhon library that enhances the user eperience by displaying
        # a progress bar in a long running operations
        #Provides visual feedbac for ongoing process

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")
        # Preprocesssing the model before feeding it to the textSummarization model

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        # Attention masks only focus on the valid  tokens. decides the weightage to assign
        # each part of the article during summarization
        #  It helps the model explore a wider range of potential summary
        # sequences while focusing on the most promising


        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
    # Finally, we decode the generated texts,
    # replace the  token, and add the decoded texts with the references to the metric.


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    #Rouge (Recall oriented understudy for gisting for gisting evaluation) is a set of
    # metrics used to evaluate the quality of automatic text summarization and machine translation.
    # It ensures how similiar the generated summaries to huma-written reference
    score = metric.compute()
    return score


In [19]:
# Different rouge metrics we need to calculate
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')
# rouge 1 = Which measures overlap of unigrams
# rouge 2: Which measures overlap of bigrams
# rougue L: which measures longest common subsequence of words between texts
# rougueLsum :This is likely a custom metric that might not be a standard
# ROUGE variant it's a combination of ROUGE-L and other metrics or
# a specific implementation within the chosen library

<ipython-input-19-5f509ba15885>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [20]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )
# This demonstrates the final steps of evaluating the performance of
# a text summarization model using ROUGE score
# his code snippet effectively converts the rouge_dict (containing metric
# names and F1 scores) into a well-organized and easily interpretable pandas DataFrame.

100%|██████████| 5/5 [00:13<00:00,  2.77s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022145,0.0,0.02181,0.022025


In [21]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [22]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")
# Helps for finetuning
# Resubability

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [23]:

#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [24]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
